In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hourly-weather-surface-brazil-southeast-region/sudeste.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
plt.rcParams["patch.force_edgecolor"] = True


In [3]:
df_original = pd.read_csv("/kaggle/input/hourly-weather-surface-brazil-southeast-region/sudeste.csv", chunksize=98000)
# df = pd.concat(df_original)

Dataset is too big so lets import in a chunk and work with the smallest one at first

In [4]:
df_coll = [df for df in df_original]

In [10]:
smallest_df = df_coll[-1].copy()

## Info On Some Columns


1. Instant Air Temperature (celsius degrees) = temp
2. Maximum Air Temperature (celsius degrees) = tmin
3. Minimum Air Temperature (celsius degrees) = tmax
4. Relative Humidity of Air (%) =hmdy
5. Maximum Relative Air Humidity (%) =hmax
6. Minimum Relative Air Humidity (%) = hmin
7. Instant Dew Point (celsius degrees) = dewp
8. Maximum Dew Point (celsius degrees)=dmax
9. Minimum Dew Point Temperature (celsius degrees) = dmin
10. Instant Air Atmospheric Pressure (millibars) =stp
11. Maximum Air Atmospheric Pressure (millibars) = smax
12. Minimum Air Atmospheric Pressure (millibars)= smin
13. Instant Wind Speed (metres per second) = wdsp
14. Wind Direction (radius degrees) = wdct
15. Wind Gust Intensity (metres per second) = gust
16. Solar radiation  =  gbrd
17. Precipitation (milimetres) = prcp
18. Elevation = elvt
19. Observation Datetime = mdct
20. Observation Date = date
21. Station number (INMET number) for the location = inme
22. The year (2000-2016) : yr
23. The month (0-12) : mo
24. The day (0-31): da
25. The hour : hr

*Not all the columns are mentioned in this list*

In [ ]:
# print(df.head())

In [11]:
print(smallest_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77168 entries, 9702000 to 9779167
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   wsid    77168 non-null  int64  
 1   wsnm    77168 non-null  object 
 2   elvt    77168 non-null  float64
 3   lat     77168 non-null  float64
 4   lon     77168 non-null  float64
 5   inme    77168 non-null  object 
 6   city    77168 non-null  object 
 7   prov    77168 non-null  object 
 8   mdct    77168 non-null  object 
 9   date    77168 non-null  object 
 10  yr      77168 non-null  int64  
 11  mo      77168 non-null  int64  
 12  da      77168 non-null  int64  
 13  hr      77168 non-null  int64  
 14  prcp    8255 non-null   float64
 15  stp     77168 non-null  float64
 16  smax    77168 non-null  float64
 17  smin    77168 non-null  float64
 18  gbrd    45016 non-null  float64
 19  temp    77168 non-null  float64
 20  dewp    77164 non-null  float64
 21  tmax    77168 non-null  flo

# Tidying Data

## Drop Elvt,Lat and Long

 Lat and long can correlate to city and province, so, remove elevation, longitude and lattitude 

In [13]:
# df.drop(['lat','lon', 'elvt'],inplace=True, axis=1)
smallest_df.drop(['lat','lon', 'elvt'], axis=1,inplace=True)

In [14]:
print(smallest_df.iloc[:,:10].head(1))
print("=="*20)
print(smallest_df.iloc[:,10:20].head(1))
print("=="*20)

print(smallest_df.iloc[:,20:].head(1))

         wsid       wsnm  inme       city prov                 mdct  \
9702000   422  ITUVERAVA  A753  Ituverava   SP  2013-06-16 16:00:00   

               date    yr  mo  da  
9702000  2013-06-16  2013   6  16  
         hr  prcp    stp   smax   smin      gbrd  temp  dewp  tmax  dmax
9702000  16   0.0  944.3  945.5  944.3  2346.659  27.9  13.7  28.2  14.7
         tmin  dmin  hmdy  hmax  hmin  wdsp   wdct  gust
9702000  27.1  13.2  42.0  46.0  40.0   3.8  332.0   9.5


In [16]:
#Drop the weather station name since its being represented as wsid


smallest_df.drop('wsnm', axis=1,inplace=True)

## Drop Multiple Time related columns

There are five columns on time of observatoin. They are mdct, date,yr, month and hour, which are same but separated into several sctions. Among them Date and Hour columns are useful and represent all in some form, so others can be dropped. The conversion of date column to datetime gave format error. So, first lets combine the yr,mo,da and hr columns to make one and then drop them

In [32]:
smallest_df['date']=pd.to_datetime(smallest_df[['yr', 'mo', 'da','hr']].rename(columns={'yr': 'year','mo': 'month','da': 'day','hr':'hour'}))

In [33]:
smallest_df.drop(['mdct','yr','da','mo'], axis= 1, inplace=True)

Reset Index as Date column and the check for duplicates data on same date in same city and remove them

In [35]:
smallest_df.set_index('date',inplace=True,drop=True)

In [37]:
smallest_df.duplicated('city')

date
2013-06-16 16:00:00    False
2013-06-16 17:00:00     True
2013-06-16 18:00:00     True
2013-06-16 19:00:00     True
2013-06-16 20:00:00     True
                       ...  
2016-09-30 19:00:00     True
2016-09-30 20:00:00     True
2016-09-30 21:00:00     True
2016-09-30 22:00:00     True
2016-09-30 23:00:00     True
Length: 77168, dtype: bool

Since there is no null values,under the assumption that all the data from various weather stations are valid and true, lets not filter the time of operation of weather stations and drop the wsids and inme, only city adn prov are enough.

In [36]:
# df.drop(['wsid','inme'], inplace=True, axis=1)
smallest_df.drop(['wsid','inme'],inplace=True, axis=1)

KeyError: "['wsid' 'inme'] not found in axis"

# Exploration and Manipulation

In [ ]:
# # print(df['wsid'].unique())
# print(df['wsid'].value_counts(normalize=True)*100)

In [ ]:
# print(df['city'].unique())
# print(df['city'].value_counts(normalize=True)*100)

In [ ]:
# df['prov'].value_counts(normalize=True).plot.pie(figsize=(8,10),autopct = '%.1f%%',labels=['Minas Gerais','São Paulo','Rio de Janeiro','Espírito Santo'])
# plt.xlabel(" ")
# plt.ylabel(" ")
# # plt.legend(['Minas Gerais','São Paulo','Rio de Janeiro','Espírito Santo'], loc=0)
# plt.title('Weather Data by Province')

In [ ]:
smallest_df.isnull().sum()